In [5]:
import numpy as np
import pandas as pd
import functools
import math
import fractions
import itertools as it
import scipy.integrate

$\textbf{Формулы Ньютона - Котеса}$

$\int_{a}^{b} f(x)dx= \sum_{i = 0}^{n - 1} f(\xi_i)(x_{i + 1} - x_{i})$

$\xi_i = \frac{x_{i+1} - x_{i}}{2}$


$\textbf{Формула трапеции}$

$\int_{a}^{b} f(x)dx= \sum_{i = 0}^{n - 1} \frac{(f(x_i) + f(x_{i + 1})}{2}(x_{i + 1} - x_{i})$

$\xi_i = \frac{x_{i+1} - x_{i}}{2}$

In [6]:
def TrapInt (X, Y):
    S = 0.
    for i in range(len(X) - 1):
        S += (Y[i] + Y[i + 1])*(X[i + 1] - X[i])/2
#        print((Y[i] + Y[i+1])*(X[i + 1] - X[i])/2)
    return S

In [7]:
#Реализация scipy, пример:
def f(x):
    return x**2
X = np.array([-1.0 + i*0.2 for i in range(11)])
Y = f(X)
print("scipy: ", scipy.integrate.trapz(Y, X))
print("TrapInt: ", TrapInt(X, Y))

scipy:  0.6799999999999999
TrapInt:  0.6799999999999999


$\textbf{Формула Симпсона}$

$\int_a^b f(x)dx \approx \sum_{i = 0}^{\frac{n}{2}}\frac{h_i}{2}(f_{2i} + 4f(x_{i + \frac{1}{2}}) + f(x_{i + 1}))$

In [8]:
#Пример реализации scipy:
f = lambda x: x*x
#Или, что тоже самое def f(x): return x**2
scipy.integrate.simps(f(np.arange(-1, 1, 0.02)), np.arange(-1, 1, 0.02))

0.6470653333333349

$\textbf{Удобный способ быстро посчитать определённый интеграл}$

In [9]:
scipy.integrate.quad(lambda x: np.sin(x), 0, np.pi)

(2.0, 2.220446049250313e-14)

$\textbf{Квадратуры Гаусса}$
$\int_{a}^{b} f(x)dx = \sum_{i = 1}^{n}c_i f(x_i)$

$\{c_i\}$ - веса квадратур (n штук)

$\{x_i\}$ - узлы квадратур (n штук)

Требуем, чтобы формула была точна для многочленов как можно большей степени

$\frac{b^k - a^k}{k + 1} = \sum_{i = 1}^{n} c_i (x_i)^k$

$\textbf{Формула Эйлера-Маклорена}$
$\int_{a}^{b}f(x)dx = \sum_{i = 0}^{n}(\frac{(f(x_i) + f(x_{i + 1}))}{2}(x_{i + 1} - x_i) - \frac{(x_{i+1} - x_{i})^2}{2}(f'(x_{i+1}) - f'(x_{i}))$

In [10]:
def EuMac(X , Y, U):#U - массив значений производной
    S = 0.
    for i in range(len(X) - 1):
        S += (Y[i] + Y[i + 1])*(X[i + 1] - X[i])/2 - ((X[i + 1] - X[i])**2)*(U[i + 1] - U[i])/2
        #print((Y[i] + Y[i+1])*(X[i + 1] - X[i])/2)
    return S

$\textbf{Экстраполяция Ричардсона}$

Пусть происходит приближённое вычисление интеграла с шагом суммирования h и с порядком точности p, то есть:

$\int_{a}^{b}f(x)dx = I(h) + c h^p$

Тогда несложно получить:
$\int_a^{b}f(x)dx = I(\frac{h}{2}) + \frac{I(\frac{h}{2}) - I(h)}{2^p - 1} + O(h^{p + 1})$

In [11]:
def R(I1, I2, p):
    return (I2 - I1)/(2**p - 1)

$\textbf{Процесс Эйткена: Пусть существует порядок p для приближённого интегрирования, тогда для порядка аппроксимации:}$

$p = \frac{1}{2}\log \left(\frac{I_{\frac{h}{2}} - I_h}{I_{\frac{h}{4}} - I_{\frac{h}{2}}}\right)$

Далее I, I2, I4 - $I(h), I(\frac{h}{2}, I(\frac{h}{4})$

In [24]:
def p(I, I2, I4):
    return np.log((I2 - I)/(I4 - I2))/np.log(2)

In [25]:
#ch^p
def EitDelt(I, I2, I4):
    return ((I2 - I)**2)/(2*I2 - I - I4)

In [26]:
EitDelt(-0.0051325,0.386161 ,0.501019)

0.5538746041743915

In [27]:
p(-0.0051325,0.386161 ,0.501019)

1.7683997993184286

In [28]:
R(0.386161, 0.501019, 2) + 0.501019

0.539305

In [17]:
X = np.array([0.+0.25*i for i in range(9)])
f = np.array([0., 0.489540, 0.647175, 0.697211, 0.642093, 0.464366, 0.130279, -0.419361, -1.294451])
scipy.integrate.simps(f, X)

0.5393055833333332

In [35]:
#Функция для вычисления остатка
def R(I, h, p):
    return (h/2 - I(h))/(2**p - 1)

$\textbf{Вычисление интегралов от функций с особенностями.}$

1.Несобственные интегралы 1,2 рода.

2.Интегралы от быстроосциллирующих функций

Пример:

$$I = \int_0^1\frac{\cos(x)}{\sqrt(x)}dx$$

Для интегрирования неограниченных функций комбинируют аналитические и численные методы.\\
1)Замена переменной

$$I = | x = t^2 | = 2\int_0^1\cos(t^2)dt$$

2)Интегрирование по частям

$$I = 2\sqrt x \cos(x)|_0^1 + 2\int_0^1\sqrt x\sin(x)dx$$ (Особенность пропала)

3)Метод явного выделения особенности Канторовича

$$I = \int_0^1\frac{\cos(x)}{\sqrt(x)}dx = \int_0^1\frac{1}{\sqrt(x)}dx + \int_0^1\frac{\cos(x) - 1}{\sqrt(x)}dx = 2\sqrt(x)|_0^1 - 2\int_0^1\frac{\sin^2(\frac{x}{2})}{\sqrt x}dx$$

4) Разложение в ряд
$$I = \int\limits_0^{\delta} \frac{\cos x}{\sqrt{x}} \, dx + \int\limits_{\delta}^{1} \frac{\cos x}{\sqrt{x}} \, dx = \int\limits_0^{\delta} \frac{1 - x^2/2 + x^4/24 + ...}{\sqrt{x}} \, dx + \int\limits_{\delta}^{1} \frac{\cos x}{\sqrt{x}} \, dx$$




Задача:
Описать алгортм вычисления методом трапеций для заданной функции.

$\int_0^{\frac{\pi}{2}}\ln (\sin (x))dx$ - с точностью $\epsilon = 10^{-4}$

Решение:

$$\int_0^\frac{\pi}{2}\ln(\sin(x))dx = \int_0^{\frac{\pi}{2}}\ln(\frac{\sin(x)}{x})dx = (x\ln(x) - x)|_0^{\frac{\pi}{2}} + \int_0^{\frac{\pi}{2}}\ln(\frac{\sin(x)}{x})dx $$ 